Ch2 p63

At the end of SS 2.12, it is suggested that it would be more efficient to avoid recomputing the partials at each step of Newton's method for a nonlinear system, doing it only after each nth step when there are n equations. Redo p59 and p62 using this modification. Compare the rate of convergence with that when the partials are recomputed at each step.

In [37]:
import numpy as np
from naf.linalg_exp import gedo, dosv, set_options
import math
import time

set_options(precision=5)

p59 Redo

In [51]:
def a_m(z):
    x = z[0]
    y = z[1]
    
    dxf1 = lambda x,y: 2.0*x + 1.0
    dyf1 = lambda x,y: -2.0*y
    dxf2 = lambda x,y: -2.0*x*math.cos(math.pow(x,2))
    dyf2 = lambda x,y: 1.0
    
    return np.array([[dxf1(x,y),dyf1(x,y)],[dxf2(x,y),dyf2(x,y)]])

def b_v(z):
    x = z[0]
    y = z[1]
    
    f1 = lambda x,y: math.pow(x,2) + x - math.pow(y,2) - 1
    f2 = lambda x,y: y - math.sin(math.pow(x,2))
    return np.array([f1(x,y),f2(x,y)])

def newton_soe(x0, a_m, b_m, tol= 0.0001, max_iter=20, verbose=False):
    """Netwon's method applied for a system of nonlinear equations
    
    Parameters:
    -----------
    x0 : 1D numpy array
        initial estimate of solution vector
    a_m : function
        function that accepts 1D numpy array of the approx. solution vector
        and returns the Jacobian matrix of the funcitons (a 2D numpy array 
        of the partial derivatives evaluated at the approx. solution vector)
    b_v : function
        function that accepts 1D numpy array of the approx. solution vector
        and returns a 2D numpy array of the functions evaluated at the approx.
        solution vector)
    tol : float, optional
        convergence tolerance. The default is 0.0001
    max_iter : integer, optional
        maximum number of iterations to perform if the solution is not converging.
        The default is 20.
    verbose : bool, optional
        outputs a summary of the solution progression at each solution step
        The default is False
        
    Returns:
    --------
    x0 : 1D numpy array
        approx. solution to the system within the specified tolerance
    dx : 1D numpy array
        solution error w.r.t. the previous approx. solution
    num_iter : integer
        number of iterations required to converge.
        
    Notes:
    ------
    Example a_m function:
    
    def a_m(z):
        x = z[0]
        y = z[1]

        dxf1 = lambda x,y: 2.0*x + 1.0
        dyf1 = lambda x,y: -2.0*y
        dxf2 = lambda x,y: -2.0*x*math.cos(math.pow(x,2))
        dyf2 = lambda x,y: 1.0

        return np.array([[dxf1(x,y),dyf1(x,y)],[dxf2(x,y),dyf2(x,y)]])
    
    Example b_m function:
    
    def b_v(z):
        x = z[0]
        y = z[1]

        f1 = lambda x,y: math.pow(x,2) + x - math.pow(y,2) - 1
        f2 = lambda x,y: y - math.sin(math.pow(x,2))
        return np.array([f1(x,y),f2(x,y)])   
    """

    num_eqs = np.size(x0)
    tol = np.full(num_eqs, 0.0001)
    dx = tol*10
    num_iter = 0
    max_iter = 20

    while np.all(abs(dx)>tol) and num_iter < max_iter:
        a = a_m(x0)

        b = -1*b_v(x0)

        lu, ov = gedo(a)
        dx = dosv(lu, ov, b)[ov]

        x0 = x0 + dx
        num_iter += 1

        if verbose:
            print(f'a: {a}')
            print(f'b: {b}')
            print(f'lu,ov: {lu},{ov}')
            print(f'dx: {dx}')
            print(f'x0: {x0}')
            print(f'num_iter: {num_iter}')
            print('\n')
                
    return x0, dx, num_iter

def newton_soem(x0, a_m, b_m, num_eqs, n, tol= 0.0001, max_iter=20, verbose=False):
    """Modified Newton's method applied to a system of nonlinear equations
    
    The modification in the solution routine is not updating the Jacobian matrix
    of partials with each iteration. Instead the same Jacobian is used to without
    update with the new approximate solution. The Jacobian is updated after n
    iterations with the first Jacobian. Using the same Jacobian for multiple 
    iteration steps saves on re-evaluation the potentially large number of 
    functions in the Jacobian matrix. This is a trade off with the number of 
    steps required to converage.
    
    Parameters:
    -----------
    x0 : 1D numpy array
        initial estimate of solution vector
    a_m : function
        function that accepts 1D numpy array of the approx. solution vector
        and returns the Jacobian matrix of the funcitons (a 2D numpy array 
        of the partial derivatives evaluated at the approx. solution vector)
    b_v : function
        function that accepts 1D numpy array of the approx. solution vector
        and returns a 2D numpy array of the functions evaluated at the approx.
        solution vector)
    num_eqs : integer
        number of equations in the system
    n : integer
        number of iterations to use the same Jacobian w/o update (a good starting
        point is the number of functions in the system)
    tol : float, optional
        convergence tolerance. The default is 0.0001
    max_iter : integer, optional
        maximum number of iterations to perform if the solution is not converging.
        The default is 20.
    verbose : bool, optional
        outputs a summary of the solution progression at each solution step.
        The default is False.
        
    Returns:
    --------
    x0 : 1D numpy array
        approx. solution to the system within the specified tolerance
    dx : 1D numpy array
        solution error w.r.t. the previous approx. solution
    num_iter : integer
        number of iterations required to converge.
        
    Notes:
    ------
    Example a_m function:
    
    def a_m(z):
        x = z[0]
        y = z[1]

        dxf1 = lambda x,y: 2.0*x + 1.0
        dyf1 = lambda x,y: -2.0*y
        dxf2 = lambda x,y: -2.0*x*math.cos(math.pow(x,2))
        dyf2 = lambda x,y: 1.0

        return np.array([[dxf1(x,y),dyf1(x,y)],[dxf2(x,y),dyf2(x,y)]])
    
    Example b_m function:
    
    def b_v(z):
        x = z[0]
        y = z[1]

        f1 = lambda x,y: math.pow(x,2) + x - math.pow(y,2) - 1
        f2 = lambda x,y: y - math.sin(math.pow(x,2))
        return np.array([f1(x,y),f2(x,y)])   
    """
    
    tol = np.full(num_eqs, 0.0001)
    dx = tol*10
    num_iter = 0
    max_iter = 20

    while np.all(abs(dx)>tol) and num_iter < max_iter:
        a = a_m(x0)
        i = 0
        
        while np.all(abs(dx)>tol) and i <= n and num_iter < max_iter:
            b = -1*b_v(x0)

            lu, ov = gedo(a)
            dx = dosv(lu, ov, b)[ov]

            x0 = x0 + dx
            
            i += 1
            num_iter += 1

            if verbose:
                print(f'a: {a}')
                print(f'b: {b}')
                print(f'lu,ov: {lu},{ov}')
                print(f'dx: {dx}')
                print(f'x0: {x0}')
                print(f'num_iter: {num_iter}')
                print('\n')

    return x0, dx, num_iter

In [52]:
start = time.time()
z1 = np.array([-2.0,1.0])
x1, dx1, ni1 = newton_soe(z1, a_m, b_v, verbose=False)
end = time.time()
print(x1, dx1, ni1, end-start)

start = time.time()
z1 = np.array([-2.0,1.0])
x1, dx1, ni1 = newton_soem(z1, a_m, b_v, 2, 2, verbose=False)
end = time.time()
print(x1, dx1, ni1, end-start)

print('\n')
start = time.time()
z2 = np.array([1.0, 1.0])
x2, dx2, ni2 = newton_soe(z2, a_m, b_v, verbose=False)
end = time.time()
print(x2, dx2, ni2, end-start)

start = time.time()
z2 = np.array([1.0, 1.0])
x2, dx2, ni2 = newton_soem(z2, a_m, b_v, 2, 2, verbose=False)
end = time.time()
print(x2, dx2, ni2, end-start)

[-1.67009  0.34514] [ 2.0e-05 -1.6e-04] 4 0.0012443065643310547
[-1.67009  0.34514] [-4.0e-05 -2.2e-04] 5 0.0010864734649658203


[0.72595 0.50295] [2.e-05 4.e-05] 4 0.0007822513580322266
[0.72596 0.50296] [5.0e-05 2.1e-04] 5 0.0008466243743896484


Discussion:
First, a note on implementation of the modified method. I noticed that instead of having a "dumb" for-loop for the inner iteration loop you could replace it with a "smarter" while-loop to potentially reduce the number of iterations if the solution converages to an acceptable tolerance while in the inner loop. I don't know if this was an intented part of the exercise and influences my discussion below. 

There doesn't appear to be a significant advantage of one method over another in this example. It is only one additional iteration for the modified method. Additionally, I am not looking at time to compute, cpu usage, or memory usage here. These may eludicate a significant advantage of one over the other. But again the problem is so small that it may be impossible to draw significant conclusions. 

I went back and timed it. A thousandth of a second or less. I don't know if I can draw conclusions from that. I think there is a way to run thousands of iterations of each of these and compare average run times which may provide some insight. Another day perhaps. My focus here is learning the how the algorithms work and how to program them.  

p62 Redo

In [40]:
def a_m(w):
    x = w[0]
    y = w[1]
    z = w[2]
    
    dxf1 = lambda x,y,z: y*z - 2*x
    dyf1 = lambda x,y,z: x*z + 2*y
    dzf1 = lambda x,y,z: x*y

    dxf2 = lambda x,y,z: y
    dyf2 = lambda x,y,z: x
    dzf2 = lambda x,y,z: -2.0*z

    dxf3 = lambda x,y,z: math.exp(x)
    dyf3 = lambda x,y,z: -1.0*math.exp(y)
    dzf3 = lambda x,y,z: 1.0

    return np.array([[dxf1(x,y,z), dyf1(x,y,z), dzf1(x,y,z)],
                     [dxf2(x,y,z), dyf2(x,y,z), dzf2(x,y,z)],
                     [dxf3(x,y,z), dyf3(x,y,z), dzf3(x,y,z)]])

def b_v(w):
    x = w[0]
    y = w[1]
    z = w[2]
    
    f1 = lambda x,y,z: x*y*z - x**2 + y**2 - 1.34
    f2 = lambda x,y,z: x*y - z**2 - 0.09
    f3 = lambda x,y,z: math.exp(x) - math.exp(y) + z - 0.41
    
    return np.array([f1(x,y,z), f2(x,y,z), f3(x,y,z)])

start = time.time()
w1 = np.array([1.0,1.0,1.0], dtype=float)
x1,dx1,ni1 = newton_soe(w1, a_m, b_v)
end = time.time()
print(x1,dx1,ni1, end-start)

print('\n')
start = time.time()
w1 = np.array([1.0,1.0,1.0], dtype=float)
x1,dx1,ni1 = newton_soem(w1, a_m, b_v, 3, 3)
end = time.time()
print(x1,dx1,ni1, end-start)

[0.90221 1.10034 0.95013] [-2.e-05 -1.e-05  0.e+00] 3 0.0010671615600585938


[0.90223 1.10034 0.95013] [-3.e-05 -6.e-05  1.e-05] 4 0.001964569091796875


Discussion:
Similar to above. No great differences in number of iterations, no real advantages seen for this type or size of problem. Possibly with the consideration of compute time, cpu or memory usage differences may be revealed. However, with this small problem I doubt any real conclusions can be reached. 

It is obvious that fewer function evaluations are needed in the modified 